# Compact Colab: Gemini + CSV + Fine-tuning (Vertex AI)
Minimal, end-to-end. Fill the TODOs, then run top to bottom.

In [ ]:

# 0) Setup
!pip -q install --upgrade google-cloud-aiplatform vertexai google-auth google-auth-oauthlib pandas pyarrow


In [ ]:

# 1) Auth and project init
from google.colab import auth
auth.authenticate_user()

import os
PROJECT_ID = "YOUR_PROJECT_ID"          # TODO: set me
LOCATION   = "us-central1"              # TODO: set region if needed
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)
print("OK: Authenticated and vertexai initialized")


In [ ]:

# 2) Load CSV
# If your CSV is in Drive, first: from google.colab import drive; drive.mount("/content/drive")
import pandas as pd

CSV_PATH = "/content/sample.csv"  # TODO: set your CSV path
df = pd.read_csv(CSV_PATH)
print("Rows:", len(df), "Cols:", len(df.columns))
df.head(3)


In [ ]:

# 3) Quick look with Gemini using a small preview (no fine-tune yet)
# We send a small slice to keep prompts short
from vertexai.generative_models import GenerativeModel, Part

preview_rows = df.head(5).to_csv(index=False)
prompt = (
    "You are a data assistant. Here is a CSV preview. "
    "Describe columns, types, potential target, and basic ideas for modeling in 5 bullets.\n\n"
    f"{preview_rows}"
)

model_name = "gemini-1.5-flash-002"  # Use a supported generation model in your region
base_model = GenerativeModel(model_name)
resp = base_model.generate_content(prompt)
print(resp.text)


In [ ]:

# 4) Build a tiny SFT JSONL from CSV rows
# Format: {"input_text": "...", "output_text": "..."}
# Here we assume a supervised text task. Adjust template as needed for your use case.
import json

# TODO: set your target column if you have one, else we do a generic Q&A
TARGET_COL = None  # e.g., "label"

def row_to_pair(row):
    if TARGET_COL and TARGET_COL in row and str(row[TARGET_COL]) != "nan":
        y = str(row[TARGET_COL])
        x = row.drop(TARGET_COL).to_dict()
        # Simple instruction style
        prompt = f"Given features {x}, predict the target."
        return {"input_text": prompt, "output_text": y}
    else:
        # Generic pattern: ask the model to summarize the row in one sentence
        x = row.to_dict()
        prompt = f"Summarize this record in one sentence: {x}"
        return {"input_text": prompt, "output_text": "A concise one sentence summary."}

# Keep this small for demo. For real tuning, build a larger set.
n = min(200, len(df))  # cap to 200 examples to stay compact
records = [row_to_pair(df.iloc[i]) for i in range(n)]
jsonl_path = "/content/train.jsonl"
with open(jsonl_path, "w", encoding="utf-8") as f:
    for r in records:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print("Wrote", jsonl_path, "with", len(records), "examples")


In [ ]:

# 5) Upload JSONL to GCS
# Requires that you have a bucket created and you have write access.
BUCKET = "gs://YOUR_BUCKET_NAME"   # TODO: set your bucket
GCS_URI = f"{BUCKET}/gemini_sft/train.jsonl"

# Use the Python storage client or gcloud storage copy. We use gcloud for brevity.
!gcloud storage cp /content/train.jsonl "$GCS_URI"
print("Uploaded to:", GCS_URI)


In [ ]:

# 6) Start a Gemini SFT tuning job
# Note: Supported base models and regions can change. If it errors, switch to a supported base model.
from vertexai.tuning import sft

BASE_MODEL = "publishers/google/models/gemini-1.5-flash-002"  # TODO: change if needed
TUNED_MODEL_DISPLAY_NAME = "gemini-sft-demo"                   # TODO: rename if needed

tuned_model = sft.train(
    model=BASE_MODEL,
    training_data_uri=GCS_URI,
    tuned_model_display_name=TUNED_MODEL_DISPLAY_NAME,
    epoch_count=3,
    batch_size=4,
    learning_rate=2e-5
)

print("Tuning started. Tuned model name: ", tuned_model.resource_name)
print("State:", tuned_model.state)  # may be PENDING or RUNNING right after submission


In [ ]:

# 7) Use the tuned model when it is ready
# You can rerun this cell later. Once state is SUCCEEDED, call it like a normal GenerativeModel.
from vertexai.generative_models import GenerativeModel

tuned_name = tuned_model.resource_name  # looks like: projects/../locations/../models/..
tuned = GenerativeModel(tuned_name)

test_prompt = "Test the tuned model on a small example input."
out = tuned.generate_content(test_prompt)
print(out.text)
